## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mehamn,NO,71.0357,27.8492,51.31,63,6,12.82,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,33.46,75,75,21.85,light snow
2,2,Fujin,CN,47.2600,132.0322,76.08,70,100,4.52,overcast clouds
3,3,Thompson,CA,55.7435,-97.8558,66.36,77,40,4.61,scattered clouds
4,4,Narsaq,GL,60.9167,-46.0500,48.42,93,100,4.34,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Fujin,CN,47.2600,132.0322,76.08,70,100,4.52,overcast clouds
3,3,Thompson,CA,55.7435,-97.8558,66.36,77,40,4.61,scattered clouds
5,5,Albany,US,42.6001,-73.9662,67.82,89,100,2.77,overcast clouds
6,6,Cape Town,ZA,-33.9258,18.4232,64.11,47,20,11.50,few clouds
7,7,Lisakovsk,KZ,52.5369,62.4936,64.17,90,90,16.53,light rain
8,8,Jamestown,US,42.0970,-79.2353,67.59,93,75,3.00,broken clouds
9,9,Ziro,IN,27.6333,93.8333,60.33,96,27,0.85,scattered clouds
10,10,Vila Franca Do Campo,PT,37.7167,-25.4333,68.41,76,94,11.07,overcast clouds
14,14,Belyy Yar,RU,53.6039,91.3903,73.29,56,0,4.47,clear sky
16,16,Rikitea,PF,-23.1203,-134.9692,73.18,81,7,17.47,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 2 to 703
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              340 non-null    int64  
 1   City                 340 non-null    object 
 2   Country              337 non-null    object 
 3   Lat                  340 non-null    float64
 4   Lng                  340 non-null    float64
 5   Max Temp             340 non-null    float64
 6   Humidity             340 non-null    int64  
 7   Cloudiness           340 non-null    int64  
 8   Wind Speed           340 non-null    float64
 9   Current Description  340 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 29.2+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna(how='any')

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Fujin,CN,76.08,overcast clouds,47.2600,132.0322,
3,Thompson,CA,66.36,scattered clouds,55.7435,-97.8558,
5,Albany,US,67.82,overcast clouds,42.6001,-73.9662,
6,Cape Town,ZA,64.11,few clouds,-33.9258,18.4232,
7,Lisakovsk,KZ,64.17,light rain,52.5369,62.4936,
8,Jamestown,US,67.59,broken clouds,42.0970,-79.2353,
9,Ziro,IN,60.33,scattered clouds,27.6333,93.8333,
10,Vila Franca Do Campo,PT,68.41,overcast clouds,37.7167,-25.4333,
14,Belyy Yar,RU,73.29,clear sky,53.6039,91.3903,
16,Rikitea,PF,73.18,clear sky,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Fujin,CN,76.08,overcast clouds,47.2600,132.0322,Xilong Hotel Fujin Shop
3,Thompson,CA,66.36,scattered clouds,55.7435,-97.8558,Burntwood Hotel
5,Albany,US,67.82,overcast clouds,42.6001,-73.9662,
6,Cape Town,ZA,64.11,few clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
7,Lisakovsk,KZ,64.17,light rain,52.5369,62.4936,Gostinitsa Lisakovsk
...,...,...,...,...,...,...,...
687,Monte Carmelo,BR,70.97,clear sky,-18.7247,-47.4986,Park hotel
695,Kuna,US,72.41,clear sky,43.4918,-116.4201,Winfield Springs
696,Domoni,KM,78.62,scattered clouds,-12.2569,44.5319,Al Djadid
699,Ferrol,ES,70.72,overcast clouds,43.4832,-8.2369,Carrís Almirante


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))